<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Manual_Practico_Machine_Learning_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manual Práctico y Didáctico de Machine Learning con Python

Este cuaderno contiene una guía completa para aprender y practicar los principales conceptos y técnicas de Machine Learning con Python. Está organizado por bloques temáticos y cada bloque incluye:

- ✅ Una **explicación breve y clara**
- 🧪 Un **ejemplo ejecutable**
- ✍️ Un **ejercicio práctico guiado**
- 📘 Una **reflexión sobre para qué sirve y cuándo se usa**

> **Sigue el orden o navega por el índice según tus intereses.**


## 1. Carga y Exploración de Datos

### ¿Qué es?

Cargar y explorar datos es el primer paso de cualquier proceso de análisis. Aquí es donde obtenemos una primera impresión de cómo están estructurados los datos, qué tipo de variables tenemos, si hay valores nulos, columnas irrelevantes, etc.

Esto se conoce también como **ETL (Extract, Transform, Load)** en entornos más profesionales.

---


### 1.1 Cargar un archivo CSV

**Código de ejemplo:**  
Cargamos un dataset desde un archivo CSV local o una URL.


In [ ]:
import pandas as pd

# Cargar archivo desde local
df = pd.read_csv("datos_calidad.csv")

# También se puede usar una URL si el archivo está en línea
# df = pd.read_csv("https://ruta-al-archivo/dataset.csv")

df.head()

**Ejercicio guiado:**  
Cambia el nombre del archivo anterior por otro CSV disponible en tu equipo o entorno, y muestra las 10 primeras filas usando `.head(10)`.


**¿Para qué sirve?**  
Este paso es fundamental para poder trabajar con datos reales. Sin esta carga inicial no es posible iniciar ningún análisis, ni visualizaciones ni modelos.


### 1.2 Exploración inicial del dataset

In [ ]:
# Ver forma del dataset
print("Dimensiones:", df.shape)

# Tipos de datos
print(df.dtypes)

# Resumen estadístico
df.describe()

**Explicación:**  
- `.shape` nos dice cuántas filas y columnas hay.
- `.dtypes` muestra el tipo de cada columna.
- `.describe()` da estadísticas básicas como media, desviación, mínimo y máximo para columnas numéricas.


### 1.3 Comprobación de valores faltantes

In [ ]:
# Comprobar nulos por columna
df.isnull().sum().sort_values(ascending=False).head(10)

**¿Para qué sirve?**  
Muchas funciones y modelos no admiten valores nulos, por lo que es importante identificarlos y decidir si se imputan (rellenan) o se eliminan.


### 1.4 Eliminar columnas irrelevantes

In [ ]:
# Supongamos que hay una columna 'ID' que no aporta valor predictivo
if 'ID' in df.columns:
    df = df.drop(columns=['ID'])


**Ejercicio guiado:**  
Busca si tu dataset contiene columnas como identificadores, fechas de carga u observaciones constantes y elimínalas del análisis.


## 2. Preparación de Datos

Antes de entrenar cualquier modelo, es fundamental preparar los datos correctamente.  
Este proceso incluye:

- **Escalar y normalizar** los datos numéricos
- **Codificar** variables categóricas
- Aplicar técnicas de **feature engineering**
- Preparar los datos para evitar fugas de información y mejorar la generalización

---


### 2.1 Escalado de variables numéricas

In [ ]:
from sklearn.preprocessing import StandardScaler

# Selección de columnas numéricas
num_cols = df.select_dtypes(include='number').columns

# Escalado estándar
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[num_cols] = scaler.fit_transform(df[num_cols])

df_scaled.head()

**¿Por qué escalar?**  
Muchos modelos (KNN, Regresión Logística, Redes Neuronales) son sensibles a la escala de las variables.  
El `StandardScaler` transforma cada variable para que tenga media 0 y desviación estándar 1.


### 2.2 Codificación de variables categóricas

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Suponemos que 'Estado' es la variable objetivo
if 'Estado' in df.columns:
    le = LabelEncoder()
    df['Estado_cod'] = le.fit_transform(df['Estado'])
    print("Clases:", list(le.classes_))
    df[['Estado', 'Estado_cod']].head()


**¿Para qué sirve?**  
Muchos modelos de machine learning necesitan que las variables categóricas estén en formato numérico.  
`LabelEncoder` convierte etiquetas como `"OK"`, `"KO"` en `0`, `1`.


### 2.3 Ingeniería de características (Feature Engineering)

In [ ]:
# Crear una nueva variable: relación entre temperatura y presión
if "Temperatura" in df.columns and "Presion" in df.columns:
    df['Temp_Pres_ratio'] = df['Temperatura'] / df['Presion']
    df[['Temperatura', 'Presion', 'Temp_Pres_ratio']].head()

**¿Para qué sirve?**  
A veces, relaciones entre variables aportan más valor predictivo que las variables originales por separado.


### 2.4 Eliminar columnas constantes o duplicadas

In [ ]:
# Eliminar columnas con un único valor
for col in df.columns:
    if df[col].nunique() == 1:
        df = df.drop(columns=[col])

**¿Para qué sirve?**  
Las columnas que no cambian no aportan información y pueden dificultar el entrenamiento o inflar el tamaño del modelo.


## 3. Modelos Supervisados

### 3.1 Regresión Lineal

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Simular datos de ejemplo
X_simple = df[["Temperatura"]]
y_simple = df["Presion"]

# Modelo
model_lr = LinearRegression()
model_lr.fit(X_simple, y_simple)
y_pred_lr = model_lr.predict(X_simple)

# Evaluación
mae = mean_absolute_error(y_simple, y_pred_lr)
mse = mean_squared_error(y_simple, y_pred_lr)
rmse = np.sqrt(mse)
r2 = r2_score(y_simple, y_pred_lr)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

**¿Para qué sirve?**  
La regresión lineal permite predecir un valor numérico continuo.  
Es útil como modelo de referencia y también para interpretar relaciones lineales entre variables.


### 3.2 Regresión Logística

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Asumimos que y_bin es binaria (0/1)
X_log = df_scaled[num_cols]
y_log = (df["Estado_cod"] == 1).astype(int) if "Estado_cod" in df else y

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_log, y_log, test_size=0.3, random_state=42)

model_log = LogisticRegression(max_iter=1000)
model_log.fit(X_train_log, y_train_log)
y_pred_log = model_log.predict(X_test_log)

print(classification_report(y_test_log, y_pred_log))

**¿Para qué sirve?**  
Clasifica instancias en dos clases (binaria) de forma simple pero efectiva.  
Suele ser el modelo base para comparar con otros más complejos.


### 3.3 Árbol de Decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier(max_depth=4)
model_dt.fit(X_train_log, y_train_log)
y_pred_dt = model_dt.predict(X_test_log)

print(classification_report(y_test_log, y_pred_dt))

**¿Para qué sirve?**  
Los árboles son fáciles de interpretar y permiten entender reglas de decisión.  
Son útiles cuando hay relaciones no lineales entre las variables.


### 3.4 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train_log, y_train_log)
y_pred_rf = model_rf.predict(X_test_log)

print(classification_report(y_test_log, y_pred_rf))

**¿Para qué sirve?**  
Es un ensemble de árboles que mejora la robustez y generalización del modelo.  
Muy usado en la industria por su rendimiento y facilidad de uso.


### 3.5 K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train_log, y_train_log)
y_pred_knn = model_knn.predict(X_test_log)

print(classification_report(y_test_log, y_pred_knn))

**¿Para qué sirve?**  
Clasifica en base a los vecinos más cercanos.  
Muy intuitivo y efectivo en datasets pequeños y bien escalados.


In [ ]:
from sklearn.neural_network import MLPClassifier

model_mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
model_mlp.fit(X_train_log, y_train_log)
y_pred_mlp = model_mlp.predict(X_test_log)

print(classification_report(y_test_log, y_pred_mlp))

**¿Para qué sirve?**  
Una red neuronal básica que permite modelar relaciones no lineales más complejas.  
Requiere datos bien escalados y más entrenamiento que otros modelos.


## 4. Modelos No Supervisados

Los modelos no supervisados aprenden a partir de datos **sin etiquetas**. Se utilizan para:
- Explorar la estructura interna de los datos
- Agrupar observaciones similares
- Reducir la dimensionalidad para visualización o mejora de modelos supervisados


### 4.1 PCA - Análisis de Componentes Principales

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Aplicar PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_log)

# Visualizar resultado
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_log, cmap='coolwarm', alpha=0.7)
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.title("Visualización PCA (2D)")
plt.grid(True)
plt.show()

**¿Para qué sirve?**  
PCA transforma las variables originales en un nuevo conjunto de variables **no correlacionadas**, que capturan la mayor parte de la varianza.
Se usa para visualización, compresión o preprocesamiento antes de modelos.


### 4.2 K-Means - Agrupamiento no supervisado

In [ ]:
from sklearn.cluster import KMeans

# Aplicar KMeans con 2 grupos
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_log)

# Visualizar con PCA
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='Accent', alpha=0.7)
plt.title("K-Means clustering con visualización PCA")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

**¿Para qué sirve?**  
K-Means permite **agrupar observaciones similares** sin necesidad de etiquetas.  
Se usa en segmentación de clientes, análisis exploratorio y detección de patrones no etiquetados.


## 5. Evaluación de Modelos

Evaluar el rendimiento de un modelo es tan importante como entrenarlo.  
Dependiendo del tipo de problema (clasificación o regresión), usaremos diferentes métricas:

- Clasificación: precisión, recall, F1, matriz de confusión, ROC, AUC
- Regresión: MAE, MSE, RMSE, R²


### 5.1 Matriz de Confusión

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test_log, y_pred_rf)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusión")
plt.show()

**¿Para qué sirve?**  
Muestra el número de aciertos y errores para cada clase.  
Ideal para saber **qué clases se confunden entre sí**.


### 5.2 Precisión, Recall, F1 Score, Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_test_log, y_pred_rf))
print("Precisión:", precision_score(y_test_log, y_pred_rf))
print("Recall:", recall_score(y_test_log, y_pred_rf))
print("F1 Score:", f1_score(y_test_log, y_pred_rf))

**¿Cuándo usar cada una?**
- `Precisión`: cuántos de los positivos predichos eran correctos
- `Recall`: cuántos de los positivos reales fueron capturados
- `F1`: equilibrio entre precisión y recall
- `Accuracy`: proporción de aciertos totales (menos útil si hay clases desbalanceadas)


### 5.3 Curva ROC y AUC

In [ ]:
from sklearn.metrics import roc_curve, auc

y_prob = model_rf.predict_proba(X_test_log)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_log, y_prob)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("Tasa de falsos positivos")
plt.ylabel("Tasa de verdaderos positivos")
plt.title("Curva ROC")
plt.legend()
plt.grid(True)
plt.show()

**¿Para qué sirve?**  
Mide la capacidad del modelo para distinguir entre clases.  
Cuanto más se acerque el AUC a 1, mejor.


### 5.4 MAE, MSE, RMSE, R²

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Usamos el modelo de regresión lineal de antes
mae = mean_absolute_error(y_simple, y_pred_lr)
mse = mean_squared_error(y_simple, y_pred_lr)
rmse = np.sqrt(mse)
r2 = r2_score(y_simple, y_pred_lr)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

**¿Qué mide cada métrica?**
- `MAE`: error absoluto medio
- `MSE`: error cuadrático medio (penaliza más errores grandes)
- `RMSE`: raíz cuadrada del MSE (en mismas unidades que la variable)
- `R²`: porcentaje de variabilidad explicada por el modelo


## 6. Técnicas de Optimización, Regularización y Redes Neuronales

Estas técnicas ayudan a mejorar el rendimiento y la generalización del modelo, especialmente cuando los datos son complejos o limitados.

---


### 6.1 Regularización L1 y L2

In [ ]:
from sklearn.linear_model import LogisticRegression

# L1 (Lasso)
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')
model_l1.fit(X_train_log, y_train_log)

# L2 (Ridge)
model_l2 = LogisticRegression(penalty='l2')
model_l2.fit(X_train_log, y_train_log)

**¿Para qué sirve?**  
- `L1`: fuerza coeficientes a cero (selección de variables)
- `L2`: reduce la magnitud de los coeficientes (reduce overfitting)


### 6.2 Búsqueda de hiperparámetros con GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5, 10]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1_macro')
grid.fit(X_train_log, y_train_log)

print("Mejores parámetros:", grid.best_params_)
print("Mejor score F1 Macro:", grid.best_score_)

**¿Para qué sirve?**  
Permite probar muchas combinaciones de hiperparámetros y seleccionar la mejor automáticamente.


### 6.3 RandomizedSearchCV (alternativa rápida a GridSearch)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import randint

param_dist = {
    'n_estimators': randint(50, 150),
    'max_depth': randint(3, 10)
}

random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=10, cv=5, random_state=42)
random_search.fit(X_train_log, y_train_log)

print("Mejores parámetros:", random_search.best_params_)

### 6.4 Rebalanceo de clases con SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train_log, y_train_log)

print("Distribución original:", Counter(y_train_log))
print("Distribución balanceada:", Counter(y_train_sm))

**¿Para qué sirve?**  
Genera muestras sintéticas para la clase minoritaria y evita el sobreajuste a la clase mayoritaria.


### 6.5 Red Neuronal con EarlyStopping

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train_log)
y_test_cat = to_categorical(y_test_log)

model_nn = Sequential()
model_nn.add(Dense(64, activation='relu', input_shape=(X_train_log.shape[1],)))
model_nn.add(Dropout(0.3))
model_nn.add(Dense(32, activation='relu'))
model_nn.add(Dense(y_train_cat.shape[1], activation='softmax'))

model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model_nn.fit(X_train_log, y_train_cat,
                       validation_split=0.2,
                       epochs=100,
                       batch_size=32,
                       callbacks=[early_stop],
                       verbose=0)

# Evaluación
loss, acc = model_nn.evaluate(X_test_log, y_test_cat, verbose=0)
print(f"Accuracy final en test: {acc:.2f}")

**¿Para qué sirve?**  
EarlyStopping evita el sobreentrenamiento parando el entrenamiento si el modelo deja de mejorar en validación.


## 7. Buenas prácticas de programación y consejos para aprender Machine Learning

Aprender a programar y resolver problemas de Machine Learning lleva tiempo. Estos consejos te ayudarán a desarrollar buenos hábitos desde el inicio y evitar errores comunes.


### 7.1 Estructura y claridad en tu código

- Escribe tu código en bloques lógicos separados por secciones.
- Usa nombres de variables descriptivos (`X_train`, `y_test`, `modelo_rf`, etc).
- Deja comentarios claros explicando qué hace cada bloque.
- Elimina código muerto o duplicado que no se esté usando.


### 7.2 Prueba tu código por partes

- No intentes resolver todo en una sola celda o paso.
- Ejecuta paso a paso: primero carga de datos, luego separación, luego modelo...
- Si algo falla, imprime los shapes de tus variables y revisa el contenido con `.head()`.


### 7.3 Aprende a leer documentación

- Usa `Shift + Tab` en notebooks o `help(función)` para ver qué hace un método.
- Lee la documentación oficial de Scikit-learn, Pandas, Seaborn y Keras.
- Copia ejemplos pequeños y prueba cambiando parámetros.


### 7.4 Reutiliza tus propios códigos como plantillas

- Guarda notebooks que ya te han funcionado.
- Usa una plantilla para cargar datos, otra para entrenar modelos, otra para gráficas.
- Esto te permitirá resolver ejercicios más rápido en el futuro y te dará confianza.


### 7.5 Ten paciencia y repite lo básico muchas veces

- No intentes dominar todos los modelos a la vez.
- Domina primero uno (RandomForest o Regresión Logística) y evalúalo bien.
- El aprendizaje real viene de repetir un problema con distintos datasets y estructuras.


## 6. Redes Neuronales: desde MLP hasta LSTM y CNN

Las redes neuronales permiten modelar relaciones no lineales complejas.  
En esta sección las construiremos de forma progresiva:

1. Red Neuronal Multicapa básica (MLP)
2. MLP más profunda con Dropout y activación ReLU
3. Red LSTM (para series temporales)
4. Red Convolucional (CNN, para datos con estructura espacial)
5. Uso de EarlyStopping para evitar overfitting


### 6.1 Red Neuronal Multicapa (MLP) Básica

**Código explicado paso a paso:**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Suponemos que y_train_log está disponible como entero
y_train_cat = to_categorical(y_train_log)
y_test_cat = to_categorical(y_test_log)

# Red con 1 capa oculta de 16 neuronas
model_basic = Sequential()
model_basic.add(Dense(16, activation='relu', input_shape=(X_train_log.shape[1],)))
model_basic.add(Dense(y_train_cat.shape[1], activation='softmax'))

model_basic.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_basic = model_basic.fit(X_train_log, y_train_cat, validation_split=0.2, epochs=30, verbose=0)

print("Entrenamiento terminado.")

**¿Qué hace este modelo?**

- Toma las variables como entrada (X)
- Usa una capa con 16 neuronas y activación ReLU
- Devuelve una predicción por clase (softmax)
- Se entrena con `categorical_crossentropy` porque es multiclase

Puedes probar aumentando las neuronas o cambiando el optimizador.


### 6.2 MLP profunda con Dropout

In [ ]:
from tensorflow.keras.layers import Dropout

model_deep = Sequential()
model_deep.add(Dense(64, activation='relu', input_shape=(X_train_log.shape[1],)))
model_deep.add(Dropout(0.3))
model_deep.add(Dense(32, activation='relu'))
model_deep.add(Dense(y_train_cat.shape[1], activation='softmax'))

model_deep.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_deep = model_deep.fit(X_train_log, y_train_cat, validation_split=0.2, epochs=50, verbose=0)

**¿Qué mejora esta red respecto a la anterior?**

- Tiene más capas y más neuronas.
- `Dropout` apaga aleatoriamente neuronas durante el entrenamiento (evita que el modelo memorice demasiado).
- Es más robusta y generaliza mejor.


### 6.3 LSTM - Red para Series Temporales

In [ ]:
import numpy as np
from tensorflow.keras.layers import LSTM, Reshape

# Creamos datos simulados: (muestras, pasos_tiempo, variables)
X_seq = np.random.rand(100, 10, 5)  # 100 series de 10 pasos y 5 variables
y_seq = to_categorical(np.random.randint(0, 2, 100))  # clasificación binaria

model_lstm = Sequential()
model_lstm.add(LSTM(32, input_shape=(10, 5)))
model_lstm.add(Dense(2, activation='softmax'))

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_lstm.fit(X_seq, y_seq, epochs=10, verbose=0)

print("Modelo LSTM entrenado para datos secuenciales.")

**¿Cuándo usar LSTM?**

- Cuando tienes secuencias temporales: sensores en el tiempo, texto, series temporales
- El modelo "recuerda" estados anteriores


### 6.4 CNN - Red Convolucional para Imágenes o Datos Espaciales

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# Simular imágenes (100 imágenes, 28x28, 1 canal)
X_img = np.random.rand(100, 28, 28, 1)
y_img = to_categorical(np.random.randint(0, 3, 100))  # 3 clases

model_cnn = Sequential()
model_cnn.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
model_cnn.add(MaxPooling2D(pool_size=(2,2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dense(3, activation='softmax'))

model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.fit(X_img, y_img, epochs=10, verbose=0)

print("Modelo CNN entrenado para datos con estructura espacial.")

**¿Para qué sirve?**

- Para imágenes, series espectrales, o cualquier dato con estructura bidimensional
- Detecta patrones locales con filtros convolucionales


### 6.5 Uso de EarlyStopping para evitar overfitting

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_es = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_log.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y_train_cat.shape[1], activation='softmax')
])

model_es.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_es.fit(X_train_log, y_train_cat, validation_split=0.2, epochs=100, batch_size=32,
             callbacks=[early_stop], verbose=0)

print("Entrenamiento con EarlyStopping finalizado.")

**¿Qué hace EarlyStopping?**

- Supervisa el rendimiento en validación
- Si no mejora durante varias épocas (`patience`), detiene el entrenamiento
- Ahorra tiempo y evita sobreajuste
